In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [3]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
import roamability as rb

downloads = 'C:/Users/balob/Downloads'

#### To get the path

In [3]:
from pathlib import Path
path_dir = Path.cwd()
print(path_dir)
path_dir

C:\Users\balob\Documents\GITLAB\RB_BD


WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

In [94]:
pwd=!echo %cd%
pwd=str(pwd)
pwd=r"c:\Users\balob\Documents\GITLAB\NOC_Roamability_com\noc\noc"
pwd.replace('\\','/',100)

'c:/Users/balob/Documents/GITLAB/NOC_Roamability_com/noc/noc'

In [ ]:
%reset -f

# Examples

### Fetch data from MS SQL DB

In [92]:
# MSSQL OCSDBREP1 (BSS)
sql_srt="""
SELECT TOP(5) * FROM USAGE_TYPE;
"""

with MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df.head(3)

,USAGE_TYPE_ID,USAGE_TYPE,USAGE_TYPE_PARENT_ID
0,1,Outgoing Call,None
1,3,Data,None
2,4,MMS,None


### Fetch data from Maria DB

In [93]:
# MariaDB BSS

sql_srt='''
SELECT MSISDN, VisitedNetworkTadig FROM TAP.GPRS_CALL LIMIT 5
'''

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn)
df.head(3)

,MSISDN,VisitedNetworkTadig
0,447700095423,BRAV1
1,972541033641,BRAV1
2,972541039874,BRAV1


In [90]:
import roamability as rb
reload(rb)
sql_srt='''
SELECT MSISDN, VisitedNetworkTadig FROM TAP.GPRS_CALL LIMIT 5
'''

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn)
df.head()

,MSISDN,VisitedNetworkTadig
0,447700095423,BRAV1
1,972541033641,BRAV1
2,972541039874,BRAV1
3,972541033641,BRAV1
4,447700095423,BRAV1


### Soap API OCS Portal

In [74]:
#%reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    #api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))
    
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934311')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
#subscriber_id

Choose connection (prod or lab):lab


Fault: The security token could not be authenticated or authorized; nested exception is org.apache.ws.security.WSSecurityException: The security token could not be authenticated or authorized

In [184]:
test = client.service.getResellerQosSpeedLimits()
test

{
    'result': {
        'code': '1',
        'description': 'Success',
        'errorUUID': None
    },
    'qosSpeedLimits': {
        'qosSpeedLimits': [
            {
                'limitName': '128kbps',
                'limitKbpsValue': 128
            }
        ]
    }
}

### Testing DMI Steering API

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
url = 'http://roamdb.roamability.com/SteeringAPI/Test'
payload = {'SponsorIMSI':'425019613990511',
           'RealIMSI':'425019613990511',
           'Mcc':'262',
           'SessionID':'1234567'}
response = requests.post(url,payload)
soup = BeautifulSoup(response.text, 'xml')
net_list = []
for net in soup.find_all('Network'):
    net_list.append([net.Rank.text,net.Mcc.text,net.Mnc.text,net.Name.text,net.Support4G.text,net.Discounted.text,net.Cost.text])
DataFrame(net_list,columns=['Rank','MCC','MNC','Operator_name','4G_Support','Discounted','Cost'])

### Send MSU

In [191]:
from bs4 import BeautifulSoup
import roamability as rb
#reload(rb)

ogt = {'p4':'48790993070',
       'partner':'97254120624',
       'sure':'447797706411',
       'tot':'66893773228',
       'porto_seguro':'550549900000',
       'rusec':'79028710069',
       'multi_byte_sponsor':'85263347864'}

rb.test_func()
#rb.print_imsi_prof('48790993070', '250350013158002', '79028710099', 'as_resp') #'as_mo'
#soup = BeautifulSoup(rb.sri4sm(48790993070, 17708103467),'xml') #250359000007598 79028710043

#rb.print_imsi_prof(ogt['rusec'], '250350013158002', '79028710099', 'as_resp') #310380608103467/19703769255

#soup = BeautifulSoup(rb.sri4sm(ogt, msisdn),'xml')
#soup = BeautifulSoup(rb.prn(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sai(ogt, dgt, imsi, node),'xml') # 0 - vlr; 16 - mme
#soup = BeautifulSoup(rb.sendSMS(ogt, imsi, msc, rb.get_info),'xml')

rb.print_imsi_prof(ogt['p4'], '260060140073744', '46705014099', 'as_resp') #310380608103467/19703769255
#soup = BeautifulSoup(rb.prn(ogt['p4'], '46705014099', '260060140073744'),'xml')
#soup

Greetings from Roamability!
<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>timeout</error>
<src_gt/>
</response> 

No info in response.


In [191]:
soup.rand.text

'60a0402193e05033f3331dd69d9b38f8'

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [66]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
any_tags  = ['#http_principle_of_operation']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\Http_module.txt
----------------------------------------------------------------------------------------------------
#http_principle_of_operation
#ocs_principle_of_operation
------------------------------
Http module не надо рестартовать, чтобы изменения в файле вступили в силу.
"Для добавления Update_Location_Relay:
1. Сконфигурировать Peer в HttpModule [на обоих OCS]
/home/roamability/config/HttpModule/routing.xml
перезапуск сервиса не требуется
2. Поставить галку "Relay LU" в Reseller"
(Владимир В., 15.10.2018, WA)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\Http_Relay.txt
----------------------------------------------------------------------------------------------------
#ocs_principle_of_operation
#http_principle_of_operation
#online_ch

### Search in ALARMS by folder names

In [12]:
import os

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('TURTS')!=-1]

c:\W_DATA_ROAM\ALARMS\MOC_Partner_TURTS_#2232821_181223
c:\W_DATA_ROAM\ALARMS\MOC_TURTS_Partner_no_CAMEL_181214_#2224717
c:\W_DATA_ROAM\ALARMS\PS_TURTS_multiple_Gy_sessions_180817_#1836812


### Read csv example

In [ ]:
#Read CSV
df_tadig = pd.read_csv('TADIG.csv',sep=';',header=None,dtype={'MCC':object,'MNC':object},
                   names=['TADIG','MCC','MNC'],skiprows=0)

### Groupby examples

In [ ]:
#Вариант №1
#Группировка с применением различных типов агрегации к колонкам groupby().agg
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']
df.groupby(['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 1 задания группировки)
df.groupby(df['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 2 задания группировки)
df.groupby(df['key1']).agg({'data1': [np.min,np.mean,np.max,np.size], 'data2': [np.min,np.mean,np.max,np.size]}) #Агрегация для каждого поля несколько функций
df.groupby(df['key1']).agg({'data1': [('f_min',np.min),('f_max',np.max)],'data2': [('f_min',np.min),('f_max',np.max)]}) #Агрегация для каждого поля несколько функций с переименованием
df.groupby(['key1','key2']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким колонкам
df.groupby(level=0).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 1 указания индекса)
df.groupby(level='ind1').agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 2 указания индекса)
df.groupby(level=[0,1]).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=[0,1]).agg(['min','max']) #Агрегация для каждого поля несколько функций
df[['data1','data2']].groupby(level=[0,1]).agg([('fmin','min'),('fmax','max')]) #Агрегация для каждого поля несколько функций с переименованием
()

In [ ]:
#Вариант №3
#Группировка .groupby + агрегирующая функция
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']

# The Best Practice
df.groupby(['key1'],as_index=False)['data1'].max()
df.groupby(level=0)['data1'].max().reset_index()

df['data1'].groupby(df['key1']).mean() #Агрегация одного поля по одному ключу
df.groupby(df['key1']).mean() #Агрегация всех полей по одному ключу
df[['data1','data2']].groupby(df['key1']).mean() #Агрегация нескольких полей по одному ключу
df[['data1','data2']].groupby([df['key1'],df['key2']]).mean() #Агрегация нескольких полей по нескольким ключам
df[['data1','data2']].groupby(level=0).mean() #Агрегация по одному индексу (вариант 1 указания индекса)
df[['data1','data2']].groupby(level='ind1').mean() #Агрегация по одному индексу (вариант 2 указания индекса)
df[['data1','data2']].groupby(level=[0,1]).mean() #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=0,axis=1).mean() #Агрегация с группировкой по столбцам
#или
grouped=df['data1'].groupby(df['key1'])
grouped.mean()
#или
df.groupby(df['key1'])['data1'].mean()
#Чтобы добавить префикс к названию строки/столбца
#df['data1'].groupby(df['key1']).mean().add_prefix('mean_')
#df[['data1','data2']].groupby(level=0,axis=1).mean()
df['data1_mean']=df.groupby('key1')['data1'].transform(np.max)
()

### Search in Allowed Lists and others

In [11]:
downloads = 'C:/Users/balob/Downloads'

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Wed_Feb_13_2019.csv'))
#df_al_lab=pd.read_csv('LAB_DMI.dmi_allowed_list_export_Wed_Oct_03_2018.csv')

#df_dmi_diam_oper=pd.read_csv(join(download,''))
df_np=pd.read_csv(join(downloads,'DMI.dmi_netpfx_export_Wed_Feb_13_2019.csv'))
#df_did=pd.read_csv('HRR.hrr_did_export_Wed_Aug_08_2018.csv')
#f_tap_ocs=pd.read_csv('tap_ocs_analysis_180817.csv')

In [63]:
#df_al[(df_al['PLMN code'].str.startswith('MKDCC', na=False)) & (df_al['Sponsor']=='')]
df_al.loc[(df_al['PLMN code'].str.startswith('RUS', na=False)) &
          (df_al['Sponsor'].str.contains('Partner_mon_worl', na=False)), ['Sponsor','PLMN code']].sort_values(by=['Sponsor','PLMN code'])

,Sponsor,PLMN code


In [12]:
str(df_np[df_np.Operator == 'TURTK'].Prefix.unique().tolist())
df_np[(df_np.Operator.str.startswith('MOZVT',na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)

,Operator,Prefix,Number/Range,SSN,MCC,MNC
1190,MOZVT,25886000,1,7,643,3
1191,MOZVT,25886000,1,149,643,3
5425,MOZVT,258860001201,0,1,643,3
5426,MOZVT,84970025886,1,7,643,3
5427,MOZVT,84970025886,1,149,643,3


In [65]:
plmn = 'ESPVV'
df1 = df_al.loc[(df_al['PLMN code'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','PLMN code']].sort_values(by=['Sponsor','PLMN code'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,PLMN code
3248,P4_telzar,ESPVV
3190,PA_Telzar_Soft,ESPVV
179,Partner,ESPVV
2707,Partner_All,ESPVV
2816,Partner_Dialoq_A,ESPVV
3128,Partner_RedT_Sof,ESPVV
1452,Partner_RedTea,ESPVV
1917,Partner_WeStream,ESPVV
1346,Partner_mon_euro,ESPVV
537,Partner_mondic,ESPVV


,Operator,Prefix,Number/Range,SSN,MCC,MNC
14,ESPVV,34607,1,7,214,1
15,ESPVV,34607,1,149,214,1
16,ESPVV,34697,1,7,214,1
17,ESPVV,34697,1,149,214,1
5764,ESPVV,3469700010,0,7,214,1
5765,ESPVV,3469700089,0,7,214,1
9245,ESPVV,3469700410,0,149,214,1
9246,ESPVV,3469700489,0,149,214,1
10165,ESPVV,34697004100,0,1,214,1


In [215]:
str(df_np[(df_np.Prefix.str.startswith('6683',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('51955',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
12787,PERN3,5195539,1,7,716,17
12789,PERN3,51955392201,0,7,716,17
12790,PERN3,51955392202,0,7,716,17
12794,PERN3,51955392203,0,149,716,17
12795,PERN3,51955392203,0,7,716,17
12796,PERN3,51955392204,0,7,716,17
12797,PERN3,51955392204,0,149,716,17
12798,PERN3,51955399136,0,149,716,17
12799,PERN3,51955399136,0,7,716,17
12800,PERN3,51955399165,0,7,716,17


### Hex to Dec conversion

In [227]:
hex_str = 'aa'
dec_str = '80'
dec_int = 80
bin_str = '1010'

# Hex to dec
print('Hex to dec')
print(int(hex_str,16))

# Dec to hex
print('Dec to hex')
print(hex(dec_int))
print('%x' % dec_int)

# Dec to bin
print('Dec to bin')
print(bin(dec_int)[2:].zfill(8))

# Bin to dec
print('Bin to dec')
print(int(bin_str,2))

# Hex to bin
print('Hex to bin')
print(bin(int(hex_str, 16))[2:].zfill(8))
print(bin(0xABC123EFFF))
f'{0xAB:0>16b}'

Hex to dec
170
Dec to hex
0x50
50
Dec to bin
01010000
Bin to dec
10
Hex to bin
10101010
1010101111000001001000111110111111111111


'0000000010101011'

In [188]:
f'{bin(2)} - {bin(61)} - {bin(6)}'
#2.61.6

'0b10 - 0b111101 - 0b110'

In [186]:
print(bin(10915))
a = '10101010100011'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10101010100011
5 - 84 - 3


### Rename files

In [ ]:
import os
import re
path = 'c:/Users/balob/Downloads/new1/'
prefix = 'PYCON'
topic = ''
os.chdir(path)
names = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for i,name in enumerate(names):
    short_name = re.sub('[!#?«»,() \+\-؟]','',name[:-4:][:50])
    if i < 10:
        number = '0'+str(i)
    else:
        number = str(i)
    new_name = '{}_{}_{}.mp4'.format(prefix,number,short_name)
    print('{} -> {}'.format(name,new_name))
    os.rename(name,new_name)

### Чтение логов с удаленного сервера SFTP (stat_GrayLog.log)

In [86]:
import paramiko
print_lines = 50
host = '172.19.11.191'
user = 'pavel'
secret = 'RzAZPjMjvujCPjU88bpevQp'
file_path_name = '/home/tracer/LOG/stat_GrayLog.log'
port = 22
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
sftp_client = client.open_sftp()
remote_file = sftp_client.open(file_path_name)
try:
    #for line in remote_file:
    for i,line in enumerate(reversed(list(remote_file))):
        if i < print_lines:
            if 'Total' in line:
                print(i,line,end='')
            #print(i,line,end='')
        else:
            break
finally:
    remote_file.close()
client.close()
# (https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python)

0 2019-01-24 05:00:45,431 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.978321
8 2019-01-23 05:00:47,552 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.941926
16 2019-01-22 05:00:49,749 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.060281
24 2019-01-21 05:00:49,224 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:44.623832
32 2019-01-20 05:00:47,066 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:45.239989
40 2019-01-19 05:00:45,864 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.761820
48 2019-01-18 05:00:46,195 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.864928


### Pricing counting

In [192]:
moc=1.5
mtc=0
text=0.25
data=7.37

tariff=[moc,mtc,text,data]

print("Price P4")
for k,v in {'1.2':1.2,'1.2*1.5':1.2*1.5,'1.2*1.5*1.15':1.2*1.5*1.15,'1.2*1.5*1.30':1.2*1.5*1.30,'2':1.2*2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])
print("\nPrice Partner")
for k,v in {'1.5':1.5,'1.5*1.15':1.5*1.15,'1.5*1.30':1.5*1.30,'2':2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])

Price P4
Price 1.2            : [1.8, 0.0, 0.3, 8.844]
Price 1.2*1.5        : [2.7, 0.0, 0.45, 13.266]
Price 1.2*1.5*1.15   : [3.105, 0.0, 0.5175, 15.2559]
Price 1.2*1.5*1.30   : [3.51, 0.0, 0.585, 17.2458]
Price 2              : [3.6, 0.0, 0.6, 17.688]

Price Partner
Price 1.5            : [2.25, 0.0, 0.375, 11.055]
Price 1.5*1.15       : [2.5875, 0.0, 0.43125, 12.71325]
Price 1.5*1.30       : [2.925, 0.0, 0.4875, 14.3715]
Price 2              : [3.0, 0, 0.5, 14.74]


## Testing

In [ ]:
import mglearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [ ]:
import requests
from bs4 import BeautifulSoup
resp = requests.get('http://www.cbr.ru/scripts/XML_daily.asp')
soup = BeautifulSoup(resp.content,'xml')
eur = soup.find(ID='R01239').Value.string
print('Курс EUR на составляет: {} RUB'.format(eur))

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
with sr.Microphone() as source:
    print('Say:')
    audio = r.listen(source)
    try:
        text = r.recognize_google(audio)
        print(f'You said: {text}')
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
    except:
        print('Sorry, I cant recognize what you said.')

In [ ]:
from gtts import gTTS 
import os
from os.path import join
downloads = 'C:/Users/balob/Downloads'
mytext = 'Привет'
language = 'en'
language = 'ru'
voice = gTTS(text=mytext, lang=language, slow=False) 
voice.save(join(downloads,'welcome.mp3')) 
os.system(join(downloads,'welcome.mp3'))

In [101]:
from bs4 import BeautifulSoup
from os.path import join
import pandas as pd
from pandas import Series, DataFrame

downloads = 'C:/Users/balob/Downloads'
file_ir21 = 'IR21_IRLDF_Three Ireland _Hutchison_ Limited_20171114_01.xml'

with open(join(downloads,file_ir21)) as inf:
    ir21 = inf.read()

xml_ir21 = BeautifulSoup(ir21,'xml')

msisdn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('MSISDN_NumberRanges'):
    for MSISDN_RangeData in MSISDN_NumberRanges.findAll('MSISDN_RangeData'):
        for NumberRange in  MSISDN_RangeData.findAll('NumberRange'):
            msisdn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'msisdn'])

nn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('GT_NumberRanges'):
    for MSISDN_RangeData in MSISDN_NumberRanges.findAll('RangeData'):
        for NumberRange in  MSISDN_RangeData.findAll('NumberRange'):
            nn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'nn'])
            
msrn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('MSRN_NumberRanges'):
    for NumberRange in  MSISDN_NumberRanges.findAll('NumberRange'):
        msrn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'msrn'])
        
e164_list = []
e164_list.extend(msisdn_list)
e164_list.extend(nn_list)
e164_list.extend(msrn_list)

for tadigs in xml_ir21.findAll('SenderTADIG'):
    tadig = tadigs.text
    print(tadig)
    
df = DataFrame(e164_list, columns = ['e164', 'type'])
df['tadig'] = tadig
df['tadig'] = 'IRLDF'

display(df)

pd.merge(df_np.loc[df_np.Operator == tadig, ['Operator', 'Prefix', 'Number/Range', 'SSN']],
         df[df.type == 'nn'], how='left', left_on=['Operator', 'Prefix'], right_on=['tadig', 'e164'])

IRLH3


,e164,type,tadig
0,35386,msisdn,IRLDF
1,35383,msisdn,IRLDF
2,35383,msisdn,IRLDF
3,35383,msisdn,IRLDF
4,35383,msisdn,IRLDF
5,35383,msisdn,IRLDF
6,35386,nn,IRLDF
7,35383,nn,IRLDF
8,35383,nn,IRLDF
9,35386,msrn,IRLDF


,Operator,Prefix,Number/Range,SSN,e164,type,tadig
0,IRLDF,35386,1,7,35386,nn,IRLDF
1,IRLDF,35386,1,149,35386,nn,IRLDF
2,IRLDF,35383,1,7,35383,nn,IRLDF
3,IRLDF,35383,1,7,35383,nn,IRLDF
4,IRLDF,35383,1,149,35383,nn,IRLDF
5,IRLDF,35383,1,149,35383,nn,IRLDF
6,IRLDF,3538600000,0,1,NaN,NaN,NaN


In [40]:
df_np.head()

,Operator,Prefix,Number/Range,SSN,MCC,MNC
0,GRCCO,3097,1,7,202,2
1,GRCCO,3097,1,149,202,2
2,GRCSH,30699,1,7,202,10
3,GRCSH,30699,1,149,202,10
4,CHEOR,4178931,1,7,228,3


In [176]:
from os.path import isfile, isdir, join, normpath
import os

folder = r"c:\Users\balob\Downloads\new3"

files = [file for file in os.listdir(folder) if isfile(join(folder, file)) and file.find('RUSEC') != -1]
files

ver4_col = ['CDR_ID',
            'USAGE_TIMESTAMP',
            'Subscriber ID',
            'IMSI',
            'ICCID',
            'MSISDN',
            'Account ID',
            'Account Name',
            'USAGE_TYPE_ID',
            'USAGE_TYPE_DESCRIPTION',
            'MCC',
            'MNC',
            'Network Name',
            'Country Name',
            'Tadig Code',
            'Destination phone number',
            'Session ID',
            'APN',
            'RAT (RadioAccessType)',
            'IMEI',
            'Download Bitrate',
            'Upload Bitrate',
            'QUANTITY',
            'QUANTITY_DESCRIPTION',
            'COST',
]

ver3_col = ['CDR_ID',
            'USAGE_TIMESTAMP',
            'Subscriber ID',
            'IMSI',
            'ICCID',
            'MSISDN',
            'Account ID',
            'Account Name',
            'USAGE_TYPE_ID',
            'USAGE_TYPE_DESCRIPTION',
            'MCC',
            'MNC',
            'Network Name',
            'Country Name',
            'Destination phone number',
            'Session ID',
            'QUANTITY',
            'QUANTITY_DESCRIPTION',
            'COST',
]

df = DataFrame(columns = ver4_col)
#df = DataFrame()
for file in files:
    df = df.append(pd.read_csv(join(folder, file), parse_dates=['USAGE_TIMESTAMP'], dtype='str', header=None, names=ver4_col))
    #df = df.append(pd.read_csv(join(folder, file), dtype='str', header=None))
    
df.to_csv(join(folder, 'joined_nrt_cdr.csv'), index=False)
df

,CDR_ID,USAGE_TIMESTAMP,Subscriber ID,IMSI,ICCID,MSISDN,Account ID,Account Name,USAGE_TYPE_ID,USAGE_TYPE_DESCRIPTION,MCC,MNC,Network Name,Country Name,Tadig Code,Destination phone number,Session ID,APN,RAT (RadioAccessType),IMEI,Download Bitrate,Upload Bitrate,QUANTITY,QUANTITY_DESCRIPTION,COST
0,400111,2019-02-07 02:41:04,1089923,250350310003966,NaN,79003950416,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,c0-10-101-210-2-EPGAM3001.roamability.com;1511265504;81249051,internet,6,352593066721778,160000000,160000000,1228186.58304,bytes,0.01124
0,416416,2019-02-07 09:28:48,1089923,250350310003966,NaN,79003950416,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,c0-10-101-210-2-EPGAM3001.roamability.com;1511265504;81249051,internet,6,352593066721778,160000000,160000000,398175.76448,bytes,0.00365
0,424647,2019-02-07 11:31:44,1089921,250350210005417,NaN,79003750417,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,d1-EPGrh.live.partner.com;1538597561;39832796;5c5c07cc-42547631840043,uinternet,6,NaN,160000000,160000000,9087116.9024,bytes,0.08666
0,429524,2019-02-07 12:39:24,1089921,250350210005417,NaN,79003750417,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,d0-EPGrh.live.partner.com;1528923558;115227101;5c5c18a3-41241140160059,uinternet,6,864004032880327,160000000,160000000,10485760,bytes,0.10000
1,429573,2019-02-07 12:40:05,1089921,250350210005417,NaN,79003750417,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,d0-EPGrh.live.partner.com;1528923558;115227101;5c5c18a3-41241140160059,uinternet,6,864004032880327,160000000,160000000,10485760,bytes,0.10000
2,429638,2019-02-07 12:40:50,1089921,250350210005417,NaN,79003750417,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,d0-EPGrh.live.partner.com;1528923558;115227101;5c5c18a3-41241140160059,uinternet,6,864004032880327,160000000,160000000,10485760,bytes,0.10000
3,429731,2019-02-07 12:42:22,1089921,250350210005417,NaN,79003750417,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,d0-EPGrh.live.partner.com;1528923558;115227101;5c5c18a3-41241140160059,uinternet,6,864004032880327,160000000,160000000,10485760,bytes,0.10000
4,429852,2019-02-07 12:43:54,1089921,250350210005417,NaN,79003750417,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,d0-EPGrh.live.partner.com;1528923558;115227101;5c5c18a3-41241140160059,uinternet,6,864004032880327,160000000,160000000,10485760,bytes,0.10000
5,429986,2019-02-07 12:46:07,1089921,250350210005417,NaN,79003750417,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,d0-EPGrh.live.partner.com;1528923558;115227101;5c5c18a3-41241140160059,uinternet,6,864004032880327,160000000,160000000,3443565.52704,bytes,0.03284
0,430261,2019-02-07 12:50:11,1089921,250350210005417,NaN,79003750417,352549,RUSEC Test SIM,33,Data,250,28,VimpelCom PJSC,Russian Federation,RUSBD,NaN,d0-EPGrh.live.partner.com;1528923558;115230353;5c5c1b36-41241140160059,uinternet,6,864004032880327,160000000,160000000,10485760,bytes,0.10000


In [2]:
def function(a, b):
    """Do X and return a list."""
    return a + b
print(help(function))
print(function.__doc__)

Help on function function in module __main__:

function(a, b)
    Do X and return a list.

None
Do X and return a list.


In [351]:
%reset -f